In [169]:
# imports
import datetime
from datetime import datetime as dttm
from datetime import date
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from geopy.geocoders import Nominatim
from sklearn.linear_model import Ridge
from datetime import timedelta
from sklearn.metrics import mean_absolute_error

In [170]:
# location name to coordinates
def coord(location):
    geolocator = Nominatim(user_agent = "predictor")
    return geolocator.geocode(location)

In [171]:
# set time period
today = date.today()
de = int(today.strftime("%d"))
me = int(today.strftime("%m"))
ye = int(today.strftime("%Y"))
start = dttm(2021, 1, 2)
end = dttm(ye, me, de)

In [172]:
# get place
city = input('Enter a city - ')
x = coord(city)
lat = int(x.latitude)
lon = int(x.longitude)
place = Point(lat, lon)
#place = Point(21.2120677, 81.3732849)

Enter a city - raipur


In [173]:
# get data
data = Daily(place, start, end)
data = data.fetch()

In [174]:
weather = pd.DataFrame(data)
weather.drop(['tavg', 'snow', 'wspd', 'wdir', 'wpgt', 'pres', 'tsun'], axis = 1, inplace = True)
print(weather)

In [175]:
weather.apply(pd.isnull).sum()

In [176]:
weather['prcp'] = weather['prcp'].fillna(0)

In [177]:
weather[pd.isnull(weather['tmax'])]

In [178]:
weather = weather.fillna(method = 'ffill')

In [179]:
weather.apply(pd.isnull).sum()

In [180]:
weather.dtypes

In [181]:
weather.index

In [182]:
#print(np.isinf(weather).values.sum())

In [183]:
weather[['tmax', 'tmin']].plot()

In [184]:
weather['prcp'].plot()

In [185]:
weather

In [186]:
weather['target'] = weather.shift(-1)['tmax']

In [187]:
weather

In [188]:
weather = weather.iloc[:-1,:].copy()

In [189]:
weather

In [190]:
reg = Ridge(alpha = 0.1)

In [191]:
predictors = ['prcp', 'tmax', 'tmin']

In [192]:
train = weather.loc[:'2022-08-31']

In [193]:
test = weather.loc['2022-09-01':]

In [194]:
reg.fit(train[predictors], train['target'])

In [195]:
predictions = reg.predict(test[predictors])

In [196]:
mean_absolute_error(test['target'], predictions)

In [197]:
combined = pd.concat([test['target'], pd.Series(predictions, index = test.index)], axis = 1)
combined.columns = ['actuals', 'predictions']

In [198]:
combined

In [199]:
combined.plot()

In [200]:
reg.coef_

In [201]:
def predictor(predictors, waether, reg):
    train = weather.loc[:'2022-08-31']
    test = weather.loc['2022-09-01':]
    reg.fit(train[predictors], train['target'])
    predictions = reg.predict(test[predictors])
    error = mean_absolute_error(test['target'], predictions)
    combined = pd.concat([test['target'], pd.Series(predictions, index = test.index)], axis = 1)
    combined.columns = ['actuals', 'predictions']
    return error, combined

In [202]:
weather['month_max'] = weather['tmax'].rolling(30).mean()

In [203]:
weather

In [204]:
weather['month_day_max'] = weather['month_max'] / weather['tmax']

In [205]:
weather['max_min'] = weather['tmax'] / weather['tmin']

In [206]:
predictors = ['prcp', 'tmax', 'tmin', 'month_max', 'month_day_max', 'max_min']

In [207]:
weather = weather.iloc[30:,:].copy()

In [208]:
error, combined = predictor(predictors, weather, reg)

In [209]:
error

In [210]:
combined.plot()

In [211]:
weather['monthly_avg'] = weather['tmax'].groupby(weather.index.month, group_keys=False).apply(lambda x : x.expanding(1).mean())

In [212]:
weather

In [213]:
weather['day_of_year_avg'] = weather['tmax'].groupby(weather.index.day_of_year, group_keys=False).apply(lambda x : x.expanding(1).mean())

In [214]:
predictors = ['prcp', 'tmax', 'tmin', 'month_max', 'month_day_max', 'max_min', 'day_of_year_avg', 'monthly_avg']

In [215]:
error, combined = predictor(predictors, weather, reg)

In [216]:
error

In [217]:
combined